In [1]:
import os, sys
from django.apps import apps as django_apps
from datetime import datetime
sys.path.append('../') # add path to project root dir
os.environ["DJANGO_SETTINGS_MODULE"] = "flourish.settings"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [6]:
schedules = CohortSchedules.objects.filter(
    child_count__isnull=False).values_list('onschedule_model', flat=True)
for onschedule_model in schedules:
    onschedule_model_cls = django_apps.get_model(onschedule_model)

    model_objs = onschedule_model_cls.objects.all()
    for model_obj in model_objs:
        if model_obj.child_subject_identifier == model_obj.subject_identifier:
            child_identifiers = CaregiverChildConsent.objects.filter(
                subject_consent__subject_identifier=model_obj.subject_identifier).values_list(
                'subject_identifier', flat=True)
            if len(set(child_identifiers)) == 1:
                model_obj.child_subject_identifier = child_identifiers[0]
                model_obj.save()